<a href="https://colab.research.google.com/github/malihe1988/Fair-SMOTE/blob/master/fairsmote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
df = files.upload()

In [ ]:
from IPython.display import Markdown
from tqdm import tqdm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,ConfusionMatrixDisplay,precision_score,recall_score,f1_score,roc_curve,precision_recall_curve,PrecisionRecallDisplay,RocCurveDisplay,auc
from sklearn.metrics import balanced_accuracy_score


from sklearn.model_selection import train_test_split


from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

from IPython.display import FileLink

In [ ]:
from __future__ import print_function, division
import pdb
import unittest
import random
from collections import Counter
import pandas as pd
import numpy as np
from scipy.spatial import distance as dist
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors as NN

def get_ngbr(df, knn):
            rand_sample_idx = random.randint(0, df.shape[0] - 1)
            parent_candidate = df.iloc[rand_sample_idx]
            ngbr = knn.kneighbors(parent_candidate.values.reshape(1,-1),3,return_distance=False)
            candidate_1 = df.iloc[ngbr[0][0]]
            candidate_2 = df.iloc[ngbr[0][1]]
            candidate_3 = df.iloc[ngbr[0][2]]
            return parent_candidate,candidate_2,candidate_3

def generate_samples(no_of_samples,df,df_name):

    total_data = df.values.tolist()
    knn = NN(n_neighbors=5,algorithm='auto').fit(df)

    for _ in range(no_of_samples):
        cr = 0.8
        f = 0.8
        parent_candidate, child_candidate_1, child_candidate_2 = get_ngbr(df, knn)
        new_candidate = []
        for key,value in parent_candidate.items():
            if isinstance(parent_candidate[key], bool):
                new_candidate.append(parent_candidate[key] if cr < random.random() else not parent_candidate[key])
            elif isinstance(parent_candidate[key], str):
                new_candidate.append(random.choice([parent_candidate[key],child_candidate_1[key],child_candidate_2[key]]))
            elif isinstance(parent_candidate[key], list):
                temp_lst = []
                for i, each in enumerate(parent_candidate[key]):
                    temp_lst.append(parent_candidate[key][i] if cr < random.random() else
                                    int(parent_candidate[key][i] +
                                        f * (child_candidate_1[key][i] - child_candidate_2[key][i])))
                new_candidate.append(temp_lst)
            else:
                new_candidate.append(abs(parent_candidate[key] + f * (child_candidate_1[key] - child_candidate_2[key])))
        total_data.append(new_candidate)

    final_df = pd.DataFrame(total_data)
    if df_name == 'Adult':
        final_df = final_df.rename(columns={0:"age",1:"education-num",2:"race",3:"sex",4:"capital-gain",5:"capital-loss",6:"hours-per-week",7:"Probability"}, errors="raise")
    if df_name == 'Compas':
        final_df = final_df.rename(columns={0:"sex",1:"age_cat",2:"race",3:"priors_count",4:"c_charge_degree",5:"Probability"}, errors="raise")
    if df_name == 'Default':
    	final_df = final_df.rename(columns={0:"ID",1:"LIMIT_BAL",2:"sex",3:"EDUCATION",4:"MARRIAGE",5:"AGE",6:"PAY_0",7:"PAY_2",8:"PAY_3",9:"PAY_4",10:"PAY_5",11:"PAY_6",12:"BILL_AMT1",13:"BILL_AMT2",14:"BILL_AMT3",15:"BILL_AMT4",16:"BILL_AMT5",17:"BILL_AMT6",18:"PAY_AMT1",19:"PAY_AMT2",20:"PAY_AMT3",21:"PAY_AMT4",22:"PAY_AMT5",23:"PAY_AMT6",24:"Probability"}, errors="raise")
    if df_name == 'German':
    	final_df = final_df.rename(columns={0:"sex",1:"age",2:"Probability",3:"credit_history=Delay",4:"credit_history=None/Paid",5:"credit_history=Other",6:"savings=500+",7:"savings=<500",8:"savings=Unknown/None",9:"employment=1-4 years",10:"employment=4+ years",11:"employment=Unemployed"}, errors="raise")
    if df_name == 'Heart':
    	final_df = final_df.rename(columns={0:"age",1:"sex",2:"cp",3:"trestbps",4:"chol",5:"fbs",6:"restecg",7:"thalach",8:"exang",9:"oldpeak",10:"slope",11:"ca",12:"thal",13:"Probability"}, errors="raise")
    if df_name == 'Bank':
        final_df = final_df.rename(columns={0:"age",1:"default",2:"balance",3:"housing",4:"loan",5:"day",6:"duration",7:"campaign",8:"pdays",9:"previous",10:"Probability"}, errors="raise")
    if df_name == 'Titanic':
        final_df = final_df.rename(columns={0:"Pclass",1:"sex",2:"Age",3:"SibSp",4:"Parch",5:"Fare",6:"Probability"}, errors="raise")
    if df_name == 'Student':
        final_df = final_df.rename(columns={0:'sex', 1:'age', 2:'Medu', 3:'Fedu', 4:'traveltime', 5:'studytime', 6:'failures',
       7:'schoolsup', 8:'famsup', 9:'paid', 10:'activities', 11:'nursery', 12:'higher',
       13:'internet', 14:'romantic', 15:'famrel', 16:'freetime', 17:'goout', 18:'Dalc', 19:'Walc',
       20:'health', 21:'absences', 22:'G1', 23:'G2', 24:'Probability'}, errors="raise")

    return final_df

In [ ]:
from sklearn.metrics import confusion_matrix

def compute_metrics(y_true, y_pred, sex, disp=True):
    tp_u, tp_p, tn_u, tn_p, fp_u, fp_p, fn_p, fn_u, N_u, N_p = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

    for true_label, pred_label, sex_value in zip(y_true, y_pred, sex):
        if sex_value == 0.0:
            N_u += 1
            if true_label == 1 and pred_label == 1:
                tp_u += 1
            elif true_label == 0 and pred_label == 0:
                tn_u += 1
            elif true_label == 0 and pred_label == 1:
                fp_u += 1
            elif true_label == 1 and pred_label == 0:
                fn_u += 1
        elif sex_value == 1.0:
            N_p += 1
            if true_label == 1 and pred_label == 1:
                tp_p += 1
            elif true_label == 0 and pred_label == 0:
                tn_p += 1
            elif true_label == 0 and pred_label == 1:
                fp_p += 1
            elif true_label == 1 and pred_label == 0:
                fn_p += 1

    # Calculate True Positive Rate (TPR), False Positive Rate (FPR), True Negative Rate (TNR), and False Negative Rate (FNR)
    tpr_u = tp_u / (tp_u + fn_u) if (tp_u + fn_u) > 0 else 0.0
    fpr_u = fp_u / (fp_u + tn_u) if (fp_u + tn_u) > 0 else 0.0
    tnr_u = tn_u / (fp_u + tn_u) if (fp_u + tn_u) > 0 else 0.0
    fnr_u = fn_u / (tp_u + fn_u) if (tp_u + fn_u) > 0 else 0.0

    tpr_p = tp_p / (tp_p + fn_p) if (tp_p + fn_p) > 0 else 0.0
    fpr_p = fp_p / (fp_p + tn_p) if (fp_p + tn_p) > 0 else 0.0
    tnr_p = tn_p / (fp_p + tn_p) if (fp_p + tn_p) > 0 else 0.0
    fnr_p = fn_p / (tp_p + fn_p) if (tp_p + fn_p) > 0 else 0.0

    # Calculate Precision, Recall, Specificity, and F1 Score
    precision_u = tp_u / (tp_u + fp_u) if (tp_u + fp_u) > 0 else 0.0
    recall_u = tp_u / (tp_u + fn_u) if (tp_u + fn_u) > 0 else 0.0
    specificity_u = tn_u / (tn_u + fp_u) if (tn_u + fp_u) > 0 else 0.0
    f1_score_u = 2 * tp_u / (2 * tp_u + fp_u + fn_u) if (2 * tp_u + fp_u + fn_u) > 0 else 0.0

    precision_p = tp_p / (tp_p + fp_p) if (tp_p + fp_p) > 0 else 0.0
    recall_p = tp_p / (tp_p + fn_p) if (tp_p + fn_p) > 0 else 0.0
    specificity_p = tn_p / (tn_p + fp_p) if (tn_p + fp_p) > 0 else 0.0
    f1_score_p = 2 * tp_p / (2 * tp_p + fp_p + fn_p) if (2 * tp_p + fp_p + fn_p) > 0 else 0.0

    # Calculate Disparate Impact
    disparate_impact = ((tp_u + fp_u) / N_u) / ((tp_p + fp_p) / N_p) if ((tp_p + fp_p) / N_p) > 0 else 0.0

    # Calculate Demo Parity
    demo_parity = ((tp_u + fp_u) / N_u) - ((tp_p + fp_p) / N_p)

    # Calculate Average Odds Difference
    average_odds_difference = ((fpr_u - fpr_p) + (tpr_u - tpr_p)) / 2

    # Calculate Equal Opportunity
    equal_opportunity = tpr_u - tpr_p

    # Calculate Accuracy
    acc = (tp_u + tp_p + tn_u + tn_p) / (tp_u + tp_p + tn_u + tn_p + fp_u + fp_p + fn_p + fn_u)


    # Calculate Balanced Accuracy

    return disparate_impact,demo_parity,average_odds_difference,equal_opportunity,ACC

In [ ]:
# Load dataset
from sklearn import preprocessing
dataset_orig = pd.read_csv('adult.data.csv')

## Drop NULL values
dataset_orig = dataset_orig.dropna()

## Drop categorical features
dataset_orig = dataset_orig.drop(['workclass','fnlwgt','education','marital-status','occupation','relationship','native-country'],axis=1)

## Change symbolics to numerics
dataset_orig['sex'] = np.where(dataset_orig['sex'] == ' Male', 1, 0)
dataset_orig['race'] = np.where(dataset_orig['race'] != ' White', 0, 1)
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == ' <=50K', 0, 1)


## Discretize age
dataset_orig['age'] = np.where(dataset_orig['age'] >= 70, 70, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 60 ) & (dataset_orig['age'] < 70), 60, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 50 ) & (dataset_orig['age'] < 60), 50, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 40 ) & (dataset_orig['age'] < 50), 40, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 30 ) & (dataset_orig['age'] < 40), 30, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 20 ) & (dataset_orig['age'] < 30), 20, dataset_orig['age'])
dataset_orig['age'] = np.where((dataset_orig['age'] >= 10 ) & (dataset_orig['age'] < 10), 10, dataset_orig['age'])
dataset_orig['age'] = np.where(dataset_orig['age'] < 10, 0, dataset_orig['age'])

In [ ]:
Predicted_Variable="Probability"
Protected_Attribute="sex"#"race"

In [ ]:
#scaling data

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = dataset_orig.drop(Predicted_Variable,axis=1)
y = dataset_orig[Predicted_Variable]

In [ ]:
# List of classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
}

In [ ]:
classifier_metrics = []


In [ ]:
def Reweighting(df_train):
    N_train = len(df_train)

    df_PP = df_train[(df_train[Protected_Attribute] == 1) & (df_train[Predicted_Variable] == 1)]
    N_PP = len(df_PP)

    df_UP = df_train[(df_train[Protected_Attribute] == 0) & (df_train[Predicted_Variable] == 1)]
    N_UP = len(df_UP)

    df_PN = df_train[(df_train[Protected_Attribute] == 1) & (df_train[Predicted_Variable] == 0)]
    N_PN = len(df_PN)

    df_UN = df_train[(df_train[Protected_Attribute] == 0) & (df_train[Predicted_Variable] == 0)]
    N_UN = len(df_UN)

    df_pri= df_train[df_train[Protected_Attribute]==1]
    N_Priviliged = len(df_pri)

    df_unpri=df_train[df_train[Protected_Attribute]==0]
    N_Un_priviliged = len(df_unpri)

    df_pos=df_train[df_train[Predicted_Variable]==1]
    N_Positive = len(df_pos)

    df_neg=df_train[df_train[Predicted_Variable]==0]
    N_Negative = len(df_neg)

    W_Positive_Priviliged = (N_Priviliged * N_Positive) / (N_train * N_PP)
    W_Positive_Un_priviliged = (N_Un_priviliged * N_Positive) / (N_train * N_UP)
    W_Negative_Priviliged = (N_Priviliged * N_Negative) / (N_train * N_PN)
    W_Negative_Un_priviliged = (N_Un_priviliged * N_Negative) / (N_train * N_UN)

    df_train.loc[(df_train[Predicted_Variable] == 1) & (df_train[Protected_Attribute] == 1), 'weights'] = W_Positive_Priviliged
    df_train.loc[(df_train[Predicted_Variable] == 1) & (df_train[Protected_Attribute] == 0), 'weights'] = W_Positive_Un_priviliged
    df_train.loc[(df_train[Predicted_Variable] == 0) & (df_train[Protected_Attribute] == 1), 'weights'] = W_Negative_Priviliged
    df_train.loc[(df_train[Predicted_Variable] == 0) & (df_train[Protected_Attribute] == 0), 'weights'] = W_Negative_Un_priviliged

    X_train = df_train.drop(["weights", Predicted_Variable], axis=1)
    y_train = df_train[Predicted_Variable]
    we2 = df_train["weights"].to_numpy()

    return X_train, y_train, we2

In [ ]:
def Uniform_sampling(df_train):

    N_train = len(df_train)

    df_PP = df_train[(df_train[Protected_Attribute] == 1) & (df_train[Predicted_Variable] == 1)]
    N_PP = len(df_PP)

    df_UP = df_train[(df_train[Protected_Attribute] == 0) & (df_train[Predicted_Variable] == 1)]
    N_UP = len(df_UP)

    df_PN = df_train[(df_train[Protected_Attribute] == 1) & (df_train[Predicted_Variable] == 0)]
    N_PN = len(df_PN)

    df_UN = df_train[(df_train[Protected_Attribute] == 0) & (df_train[Predicted_Variable] == 0)]
    N_UN = len(df_UN)

    df_pri= df_train[df_train[Protected_Attribute]==1]
    N_Priviliged = len(df_pri)

    df_unpri=df_train[df_train[Protected_Attribute]==0]
    N_Un_priviliged = len(df_unpri)

    df_pos=df_train[df_train[Predicted_Variable]==1]
    N_Positive = len(df_pos)

    df_neg=df_train[df_train[Predicted_Variable]==0]
    N_Negative = len(df_neg)

    W_Positive_Priviliged = (N_Priviliged * N_Positive) / (N_train * N_PP)
    W_Positive_Un_priviliged = (N_Un_priviliged * N_Positive) / (N_train * N_UP)
    W_Negative_Priviliged = (N_Priviliged * N_Negative) / (N_train * N_PN)
    W_Negative_Un_priviliged = (N_Un_priviliged * N_Negative) / (N_train * N_UN)

    S_PP = round(W_Positive_Priviliged * N_PP)
    S_UP = round(W_Positive_Un_priviliged * N_UP)
    S_PN = round(W_Negative_Priviliged * N_PN)
    S_UN = round(W_Negative_Un_priviliged * N_UN)

    sample_df_PP = df_PP.sample(S_PP, replace=False)
    sample_df_UP = df_UP.sample(S_UP, replace=True)
    sample_df_PN = df_PN.sample(S_PN, replace=True)
    sample_df_UN = df_UN.sample(S_UN, replace=False)

    list_concat = [sample_df_PP, sample_df_UP, sample_df_PN, sample_df_UN]
    sample_df = pd.concat(list_concat)
    X_train_s = sample_df.drop([Predicted_Variable], axis=1)
    y_train_s = sample_df[Predicted_Variable]

    return X_train_s, y_train_s

In [ ]:
######to pick samples close to decision boundry

def UP_sampling(df, num_samples):
    num_samples = abs(num_samples)  # Convert num_samples to its absolute value

    total_rows = len(df)
    sampled_df = pd.DataFrame()  # Initialize an empty DataFrame to store sampled data

    while len(sampled_df) < num_samples:
        remaining_samples = num_samples - len(sampled_df)

        # If remaining samples needed are greater than the total rows, duplicate the entire DataFrame
        if remaining_samples >= total_rows:
            sampled_df = pd.concat([sampled_df, df], ignore_index=True)
        else:
            # Otherwise, concatenate a subset of rows from the top of the DataFrame
            sampled_df = pd.concat([sampled_df, df.head(remaining_samples)], ignore_index=True)

    return sampled_df

In [ ]:
def preferential_samples(X_train,y_train):

    df_train = pd.concat([X_train, y_train], axis=1)

    #scaled_X_train_ranker = scaler.fit_transform(X_train)
    ranker_model = LogisticRegression()
    ranker_model.fit(X_train, y_train)
    positive_probs_R = ranker_model.predict_proba(X_train)[:, 1]
    df_train["p_probablity"]=positive_probs_R

    N_train = len(df_train)

    df_PP = df_train[(df_train[Protected_Attribute] == 1) & (df_train[Predicted_Variable] == 1)]
    N_PP = len(df_PP)

    df_UP = df_train[(df_train[Protected_Attribute] == 0) & (df_train[Predicted_Variable] == 1)]
    N_UP = len(df_UP)

    df_PN = df_train[(df_train[Protected_Attribute] == 1) & (df_train[Predicted_Variable] == 0)]
    N_PN = len(df_PN)

    df_UN = df_train[(df_train[Protected_Attribute] == 0) & (df_train[Predicted_Variable] == 0)]
    N_UN = len(df_UN)

    df_pri= df_train[df_train[Protected_Attribute]==1]
    N_Priviliged = len(df_pri)

    df_unpri=df_train[df_train[Protected_Attribute]==0]
    N_Un_priviliged = len(df_unpri)

    df_pos=df_train[df_train[Predicted_Variable]==1]
    N_Positive = len(df_pos)

    df_neg=df_train[df_train[Predicted_Variable]==0]
    N_Negative = len(df_neg)

    W_Positive_Priviliged = (N_Priviliged * N_Positive) / (N_train * N_PP)
    W_Positive_Un_priviliged = (N_Un_priviliged * N_Positive) / (N_train * N_UP)
    W_Negative_Priviliged = (N_Priviliged * N_Negative) / (N_train * N_PN)
    W_Negative_Un_priviliged = (N_Un_priviliged * N_Negative) / (N_train * N_UN)

    S_PP = round(W_Positive_Priviliged * N_PP)
    S_UP = round(W_Positive_Un_priviliged * N_UP)
    S_PN = round(W_Negative_Priviliged * N_PN)
    S_UN = round(W_Negative_Un_priviliged * N_UN)

    PP=df_PP.sort_values(by="p_probablity", axis=0, ascending=True, inplace=False, key=None)
    UP=df_UP.sort_values(by="p_probablity", axis=0, ascending=True, inplace=False, key=None)
    UN=df_UN.sort_values(by=['p_probablity'], axis=0, ascending=False, inplace=False, key=None)
    PN=df_PN.sort_values(by=['p_probablity'], axis=0, ascending=False, inplace=False, key=None)


    sample_UP = UP_sampling(UP, S_UP)

    sample_PN = UP_sampling(PN, S_PN)

    sample_UN = UN.tail(S_UN)
    sample_PP = PP.tail(S_PP)


    list_concat = [sample_UP,sample_UN,sample_PP,sample_PN]
    new_df = pd.concat(list_concat)

    X_train_ps = new_df.drop([Predicted_Variable,"p_probablity"], axis=1)
    y_train_ps = new_df[Predicted_Variable]

    return X_train_ps, y_train_ps

In [ ]:
def Fair_Smote(df_train):

    N_train = len(df_train)

    df_PP = df_train[(df_train[Protected_Attribute] == 1) & (df_train[Predicted_Variable] == 1)]
    N_PP = len(df_PP)

    df_UP = df_train[(df_train[Protected_Attribute] == 0) & (df_train[Predicted_Variable] == 1)]
    N_UP = len(df_UP)

    df_PN = df_train[(df_train[Protected_Attribute] == 1) & (df_train[Predicted_Variable] == 0)]
    N_PN = len(df_PN)

    df_UN = df_train[(df_train[Protected_Attribute] == 0) & (df_train[Predicted_Variable] == 0)]
    N_UN = len(df_UN)


    # Calculate maximum count
    maximum = max(N_PP,N_UP,N_PN,N_UN)

    # Calculate number of samples to be generated for each class and protected attribute combination
    df_PP_to_be_increased = maximum - N_PP
    df_UP_to_be_increased = maximum - N_UP
    df_PN_to_be_increased = maximum - N_PN
    df_UN_to_be_increased = maximum - N_UN

    # Generate synthetic samples
    df_new_PP = generate_samples(df_PP_to_be_increased, df_PP, 'Adult')
    df_new_UP = generate_samples(df_UP_to_be_increased, df_UP, 'Adult')
    df_new_PN = generate_samples(df_PN_to_be_increased, df_PN, 'Adult')
    df_new_UN = generate_samples(df_UN_to_be_increased, df_UN, 'Adult')

    # Combine the resampled data
    df_train_resampled = pd.concat([df_new_PP, df_new_UP, df_new_PN, df_new_UN])
    X_train_resampled = df_train_resampled.drop(Predicted_Variable, axis=1)
    y_train_resampled = df_train_resampled[Predicted_Variable]

    return X_train_resampled, y_train_resampled

In [ ]:
#################################
#its results is different with when run ps alone
###########################

n_splits = 10

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    accuracy_original_list = []
    balanced_original_list = []
    disparate_impact_original_list = []
    demo_parity_original_list = []
    average_odds_difference_original_list = []
    equal_opportunity_original_list = []

    accuracy_reweight_list = []
    balanced_reweight_list = []
    disparate_impact_reweight_list = []
    demo_parity_reweight_list = []
    average_odds_difference_reweight_list = []
    equal_opportunity_reweight_list = []

    accuracy_sample_list = []
    balanced_sample_list = []
    disparate_impact_sample_list = []
    demo_parity_sample_list = []
    average_odds_difference_sample_list = []
    equal_opportunity_sample_list = []

    accuracy_ps_list = []
    balanced_ps_list = []
    disparate_impact_ps_list = []
    demo_parity_ps_list = []
    average_odds_difference_ps_list = []
    equal_opportunity_ps_list = []


    accuracy_fs_list = []
    balanced_fs_list = []
    disparate_impact_fs_list = []
    demo_parity_fs_list = []
    average_odds_difference_fs_list = []
    equal_opportunity_fs_list = []


    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=101)

    # Perform cross-validation
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        ##########################################baseline
        X_train_original = X_train
        X_test_original = X_test
        # Fit classifier with original data

        # Scale the data
        X_train_scaled_original = scaler.fit_transform(X_train_original)
        X_test_scaled_original = scaler.transform(X_test_original)

        clf.fit(X_train_scaled_original, y_train)

        # Predict on the test set
        y_pred = clf.predict(X_test_scaled_original)

        # Compute metrics for original data
        disparate_impact, demo_parity, average_odds_difference, equal_opportunity, acc = compute_metrics(y_test, y_pred, X_test[Protected_Attribute])

        # Append metrics to respective lists
        disparate_impact_original_list.append(disparate_impact)
        demo_parity_original_list.append(demo_parity)
        average_odds_difference_original_list.append(average_odds_difference)
        equal_opportunity_original_list.append(equal_opportunity)

        # Compute accuracy scores
        accur_orig = accuracy_score(y_test, y_pred)
        balanced_orig = balanced_accuracy_score(y_test, y_pred)

        # Append computed accuracy scores
        accuracy_original_list.append(accur_orig)
        balanced_original_list.append(balanced_orig)


        #################################################### Applying reweighting
        X_train_reweight, y_train_reweight, we2 = Reweighting(pd.concat([X_train, y_train], axis=1))

        # Scale the data
        X_train_scaled_reweight = scaler.fit_transform(X_train_reweight)
        X_test_scaled = scaler.transform(X_test)

        # Fit classifier with reweighted data
        clf.fit(X_train_scaled_reweight, y_train_reweight, sample_weight=we2)

        # Predict on the test set
        y_pred = clf.predict(X_test_scaled)

        # Compute metrics for reweighted data
        disparate_impact, demo_parity, average_odds_difference, equal_opportunity, acc = compute_metrics(y_test, y_pred, X_test[Protected_Attribute])

        # Append metrics to respective lists
        disparate_impact_reweight_list.append(disparate_impact)
        demo_parity_reweight_list.append(demo_parity)
        average_odds_difference_reweight_list.append(average_odds_difference)
        equal_opportunity_reweight_list.append(equal_opportunity)

        # Compute accuracy scores
        accur_re = accuracy_score(y_test, y_pred)
        balanced_re = balanced_accuracy_score(y_test, y_pred)

        # Append computed accuracy scores
        accuracy_reweight_list.append(accur_re)
        balanced_reweight_list.append(balanced_re)

        ####################################### Applying US_samples
        X_train_sample, y_train_sample = Uniform_sampling(pd.concat([X_train, y_train], axis=1))

        # Scale the data
        X_train_scaled_sample = scaler.fit_transform(X_train_sample)

        # Fit classifier with sampled data
        clf.fit(X_train_scaled_sample, y_train_sample)

        # Predict on the test set
        y_pred = clf.predict(X_test_scaled)

        # Compute metrics for sampled data
        disparate_impact, demo_parity, average_odds_difference, equal_opportunity, acc = compute_metrics(y_test, y_pred, X_test[Protected_Attribute])

        # Append metrics to respective lists
        disparate_impact_sample_list.append(disparate_impact)
        demo_parity_sample_list.append(demo_parity)
        average_odds_difference_sample_list.append(average_odds_difference)
        equal_opportunity_sample_list.append(equal_opportunity)

        # Compute accuracy scores
        accur_us = accuracy_score(y_test, y_pred)
        balanced_us = balanced_accuracy_score(y_test, y_pred)

        # Append computed accuracy scores
        accuracy_sample_list.append(accur_us)
        balanced_sample_list.append(balanced_us)

        ####################################### Applying preferential sampling
        X_train_ps, y_train_ps = preferential_samples(X_train, y_train)

        # Scale the data
        X_train_scaled_ps = scaler.fit_transform(X_train_ps)

        # Fit classifier with preferentially sampled data
        clf.fit(X_train_scaled_ps, y_train_ps)

        # Predict on the test set
        y_pred = clf.predict(X_test_scaled)

        # Compute metrics for preferentially sampled data
        disparate_impact, demo_parity, average_odds_difference, equal_opportunity, acc = compute_metrics(y_test, y_pred, X_test[Protected_Attribute])

        # Append metrics to respective lists
        disparate_impact_ps_list.append(disparate_impact)
        demo_parity_ps_list.append(demo_parity)
        average_odds_difference_ps_list.append(average_odds_difference)
        equal_opportunity_ps_list.append(equal_opportunity)

        # Compute accuracy scores
        accur_ps = accuracy_score(y_test, y_pred)
        balanced_ps = balanced_accuracy_score(y_test, y_pred)

        # Append computed accuracy scores
        accuracy_ps_list.append(accur_ps)
        balanced_ps_list.append(balanced_ps)


        ####################################### Applying Fair_smote
        X_train_Fair_SMOTE, y_train_Fair_SMOTE = Fair_Smote(pd.concat([X_train, y_train], axis=1))

        # Scale the data
        X_train_scaled_FS = scaler.fit_transform(X_train_Fair_SMOTE)

        # Fit classifier with sampled data
        clf.fit( X_train_scaled_FS, y_train_Fair_SMOTE)

        # Predict on the test set
        y_pred = clf.predict(X_test_scaled)

        # Compute metrics for sampled data
        disparate_impact, demo_parity, average_odds_difference, equal_opportunity, acc = compute_metrics(y_test, y_pred, X_test[Protected_Attribute])

        # Append metrics to respective lists
        disparate_impact_fs_list.append(disparate_impact)
        demo_parity_fs_list.append(demo_parity)
        average_odds_difference_fs_list.append(average_odds_difference)
        equal_opportunity_fs_list.append(equal_opportunity)

        # Compute accuracy scores
        accur_fs = accuracy_score(y_test, y_pred)
        balanced_fs = balanced_accuracy_score(y_test, y_pred)

        # Append computed accuracy scores
        accuracy_fs_list.append(accur_fs)
        balanced_fs_list.append(balanced_fs)

    ######################## Compute average metrics across folds for original data
    accuracy_original = np.median(accuracy_original_list)
    balanced_accuracy_original = np.median(balanced_original_list)
    disparate_impact_original = np.median(disparate_impact_original_list)
    demo_parity_original = np.median(demo_parity_original_list)
    average_odds_difference_original = np.median(average_odds_difference_original_list)
    equal_opportunity_original = np.median(equal_opportunity_original_list)

    ######################### Compute average metrics across folds for reweighted data
    accuracy_reweight = np.median(accuracy_reweight_list)
    balanced_accuracy_reweight = np.median(balanced_reweight_list)
    disparate_impact_reweight = np.median(disparate_impact_reweight_list)
    demo_parity_reweight = np.median(demo_parity_reweight_list)
    average_odds_difference_reweight = np.median(average_odds_difference_reweight_list)
    equal_opportunity_reweight = np.median(equal_opportunity_reweight_list)

    ########################## Compute average metrics across folds for sampled data
    accuracy_sample = np.median(accuracy_sample_list)
    balanced_accuracy_sample = np.median(balanced_sample_list)
    disparate_impact_sample = np.median(disparate_impact_sample_list)
    demo_parity_sample = np.median(demo_parity_sample_list)
    average_odds_difference_sample = np.median(average_odds_difference_sample_list)
    equal_opportunity_sample = np.median(equal_opportunity_sample_list)

    ############################ Compute average metrics across folds for preferentially sampled data
    accuracy_ps = np.median(accuracy_ps_list)
    balanced_accuracy_ps = np.median(balanced_ps_list)
    disparate_impact_ps = np.median(disparate_impact_ps_list)
    demo_parity_ps = np.median(demo_parity_ps_list)
    average_odds_difference_ps = np.median(average_odds_difference_ps_list)
    equal_opportunity_ps = np.median(equal_opportunity_ps_list)

        ############################ Compute average metrics across folds for fair-smote sampled data
    accuracy_fs = np.median(accuracy_fs_list)
    balanced_accuracy_fs = np.median(balanced_fs_list)
    disparate_impact_fs = np.median(disparate_impact_fs_list)
    demo_parity_fs = np.median(demo_parity_fs_list)
    average_odds_difference_fs = np.median(average_odds_difference_fs_list)
    equal_opportunity_fs = np.median(equal_opportunity_fs_list)

    # Append metrics to the classifier_metrics list
    classifier_metrics.append([clf_name + " (Baseline)", accuracy_original, balanced_accuracy_original, disparate_impact_original, demo_parity_original, average_odds_difference_original, equal_opportunity_original])
    classifier_metrics.append([clf_name + " (Reweighting)", accuracy_reweight, balanced_accuracy_reweight, disparate_impact_reweight, demo_parity_reweight, average_odds_difference_reweight, equal_opportunity_reweight])
    classifier_metrics.append([clf_name + " (Uniform Sampling)", accuracy_sample, balanced_accuracy_sample, disparate_impact_sample, demo_parity_sample, average_odds_difference_sample, equal_opportunity_sample])
    classifier_metrics.append([clf_name + " (Preferential Sampling)", accuracy_ps, balanced_accuracy_ps, disparate_impact_ps, demo_parity_ps, average_odds_difference_ps, equal_opportunity_ps])
    classifier_metrics.append([clf_name + " (Fair_SMOTE)", accuracy_fs, balanced_accuracy_fs, disparate_impact_fs, demo_parity_fs, average_odds_difference_fs, equal_opportunity_fs])

# Create a DataFrame
metrics_df = pd.DataFrame(classifier_metrics, columns=['Classifier', 'Accuracy', 'Balanced Accuracy', 'Disparate Impact', 'Demographic Parity', 'Average Odds Difference', 'Equal Opportunity'])

# Save the DataFrame to an Excel file
metrics_df.to_excel('classifier_base_re_us_ps_fs_metrics.xlsx', index=False)

# Create a link for downloading the Excel file
FileLink(r'classifier_base_re_us_ps_fs_metrics.xlsx')